In [ ]:
from netgen.occ import *
from ngsolve import *
import netgen.meshing as meshing
from ngsolve.webgui import Draw
from ngbem import *

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(3)
n = specialcf.normal(3)
#mesh = Mesh(unit_cube.GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(3)

In [ ]:
fesl2 = SurfaceL2(mesh, order=1, dual_mapping=True)
ul2,vl2 = fesl2.TnT()

fesh1 = SurfaceL2(mesh, order=1)
uh1, vh1 = fesh1.TnT()

a = BilinearForm(uh1*vl2*ds).Assemble()
#print (a.mat)

In [ ]:
fesHCurl = HCurl(mesh, order=0)
uHtest,vHtest = fesHCurl.TnT() # H(curl_Gamma) conform spaces
print(fesHCurl.ndof)

M = BilinearForm( (uHtest.Trace()* vHtest.Trace() ) * ds(bonus_intorder=3), check_unused=True).Assemble() # <Hcurl, Hcurl>
for k in range(0,fesHCurl.ndof):
    for l in range(0, fesHCurl.ndof):
        if(abs(M.mat[l,k]) > 1e-15):
            print(l,k, M.mat[l,k])       

In [ ]:
Minv = M.mat.Inverse() 

In [ ]:
fesHCurl = HCurl(mesh, order=0)
uHtest,vHtest = fesHCurl.TnT() # H(curl_Gamma) conform spaces
print(fesHCurl.ndof)

M = BilinearForm( (Cross(n,uHtest.Trace())* vHtest.Trace() ) * ds(bonus_intorder=3), check_unused=True).Assemble() # <Hdiv, Hcurl> metric invariant
#for k in range(0,fesHCurl.ndof):
    #for l in range(0, fesHCurl.ndof):
    #    if(abs(M.mat[l,k]) > 1e-15):
    #        print(l,k, M.mat[l,k])
Minv = M.mat.Inverse() 

In [ ]:
fesHDiv = HDivSurface(mesh, order=0)
print(fesHDiv.ndof)
#print(fesHDiv.FreeDofs())
uHdiv,vHdiv = fesHDiv.TnT() # H(div_Gamma) conform spaces
M = BilinearForm( (uHdiv.Trace() * vHdiv.Trace() +  div(uHdiv).Trace() * div(vHdiv).Trace() )  * ds(bonus_intorder=3), check_unused=True).Assemble() # <Hcurl, Hdiv>
#for k in range(0,fesHDiv.ndof):
#    for l in range(0, fesHDiv.ndof):
#        if(abs(M.mat[l,k]) > 1e-15):
#            print(l,k, M.mat[l,k])      

In [ ]:
Minv = M.mat.Inverse(freedofs=fesHDiv.FreeDofs()) 

In [ ]:
fesHDiv = HDivSurface(mesh, order=0)
print(fesHDiv.ndof)
#print(fesHDiv.FreeDofs())
uHdiv,vHdiv = fesHDiv.TnT() # H(div_Gamma) conform spaces
M = BilinearForm( (Cross(uHdiv.Trace(),n) * vHdiv.Trace())  * ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv>
#for k in range(0,fesHDiv.ndof):
#    for l in range(0, fesHDiv.ndof):
#        if(abs(M.mat[l,k]) > 1e-15):
#            print(l,k, M.mat[l,k])
Minv = M.mat.Inverse() 